In [ ]:
import mne, os, glob, pickle, matplotlib, itertools
import numpy as np
import pandas as pd
mne.set_log_level(verbose=False)
matplotlib.use('Qt5Agg') #ipython
#%matplotlib inline

In [ ]:
ROOT = '/Users/alr664/Desktop/multiple_affix'
os.chdir(ROOT)
subjects_dir = ROOT + '/mri/'
meg = ROOT + '/data/'
logs = ROOT + '/logs/'

#file search
exp = 'MA'

subjects = ["A0394", "A0421", "A0446", "A0451", "A0468", "A0484", "A0495", "A0502", "A0503", "A0508", 
                "A0509", "A0512", "A0513", "A0514", "A0516", "A0517", "A0518", "A0519", "A0520", "A0521", 
                "A0522", "A0523", "A0524", "A0525"]

#manually set subj and date here
subj = ''
date = ''

event_ids = {
    '0Suff w/ Lat': 1,
    '0Suff w/o Lat.': 2,
    '0Suff NW': 4,
    '1Suff w/ Lat.': 11,
    '1Suff w/o Lat.': 12,
    '1Suff PseudoStemNW': 14,
    '1Suff RealStemNW': 15,
    '2Suff w/ Lat.': 21,
    '2Suff w/o Lat.': 22,
    '2Suff Composite': 23,
    '2Suff PseudoStemNW': 24,
    '2Suff RealStemNW': 25
}

#'practice': 99

nonwords = [4, 14, 15, 24] 
realwords = [1, 2, 11, 12, 21, 22, 23, 25] 


# --- Epochs and timing
tmin = -0.1
tmax = 1.4

baseline = (-0.1,0)
decim = 1

# --- Source Reconstruction
fixed = True    #True for Signed Data (+/-), False for Unsigned Data
SNR = 3         # 3 for evoked, 2 for single-trial/epochs
lambda2 = 1.0 / SNR ** 2.0

you'll need to make sure each logfile matches the events length. 

sometimes, the experiment is paused and resumes, which sends two triggers for the same event.
there is a trigger marking the start of the experiment and one for each trial.

events have this format:

In [ ]:
events = [[  23573       0     164]
            [  27939       0     164]
            [  33189       0     164]
            ...
            [2473938       0     165]
            [2477054       0     165]
            [2480253       0     165]]

 index [0] is the timepoint in ms,
 you can ignore index [1], it tells us the preceding event's trigger,
 index [2] tells us the current event's trigger but you can keep or change this depending on how you want to access the trial metadata

In [ ]:
print(subj + '...')
raw = mne.io.read_raw_fif('meg/%s/%s_%s_ICA-raw.fif' %(subj, subj, exp), preload=True)
events = mne.find_events(raw, min_duration=0.002)

info = raw.info
picks_meg = mne.pick_types(info, meg=True, eeg=False, eog=False, stim=False)
pickle.dump(info, open('meg/%s/%s_info.pickled' %(subj,subj), 'wb'))

event_id = {'all':1}

events_test = events

# print('Overwriting events with logfile...')
# events_test[:,1] = 0
# events_test[:,2] = 1

print("Creating epochs and metadata")
if not os.path.exists('meg/%s/epochs' %subj):
        os.makedirs('meg/%s/epochs' %subj)

logfile_dir = f'logs/{subj}_logfile.csv'
logfile = pd.read_csv(logfile_dir)  #metadata=logfile

epochs = mne.Epochs(raw, events_test, event_id=event_ids, tmin=tmin, tmax=tmax, baseline=baseline, picks=picks_meg, decim=decim, metadata=logfile, reject_by_annotation=True, preload=True)
epochs.save('meg/%s/epochs/%s_%s-epo.fif'%(subj,subj, exp), overwrite=True)
print('Done.')

In [ ]:
for subj in subjects:
    print('Rejecting epochs for %s'%(subj)) # reject epochs with any signal above threshold 3e-10
    cache_dir = ROOT + '/meg/%s/cache/' %(subj)
    epochs = mne.read_epochs('meg/%s/epochs/%s_rejection-epo.fif'%(subj,subj))
    n_init_epochs = len(epochs)
    reject = {'mag': 3e-10}
    epochs.drop_bad(reject)
    #epochs_rej = epochs[rejs]
    print(epochs)
    
    print ('Saving epochs to file...')
    n_good_epochs = len(epochs)
    n_epochs_rej = n_init_epochs - n_good_epochs

    #write log of number of epochs dropped
    file = open(cache_dir + subj + '_bad_epochs.txt','w')
    n = str(n_epochs_rej)
    file.write(n+"\n")
    file.close()

    #save only good epochs
    epochs.save('meg/%s/epochs/%s_rejection-epo.fif' %(subj,subj),overwrite=True)
    print ('Done.')

you can also manually reject epochs that are noisy but fall below the threshold.


you'll need to use eelbrain in ipython. activate the eelbrain environment, then type "eelbrain" to open a specific ipython profile 


then load in dependencies and use the GUI to reject trials *if needed* 

In [ ]:
# in 'eelbrain' ipython environment only

subj = input('subject: ')
cache_dir = ROOT + '/meg/%s/cache/' %(subj)

epochs = mne.read_epochs('meg/%s/epochs/%s_rejection-epo.fif'%(subj,subj))
n_init_epochs = len(epochs)
eelbrain.gui.select_epochs(epochs, vlim=3e-12)